# Injury Status Extraction

This notebook os meant to extract all the injury status related information from the Notes column in the injury_list table.

**Status types extracted:**
- `DTD` (Day-to-Day, "day-to-day" written out, or "game-to-game", or "days")
- `out-for-season` (out for season, remainder of season, rest of season, entire season)
- `out-indefinitely` (out indefinitely or out indefinite. Uncertain when the player comes back)
- `out-for-weeks` (if "weeks" appears in Notes. Player is certainly out for X number of weeks.)
- `out-for-months` (if "months" appears in Notes. Player is certainly out for X number of months.)

Output is saved to `Status_IL_movement.csv` without modifying the original file.
The status field in the injury_list table is also updated directly when running this notebook

In [11]:
import pandas as pd
import sqlite3
import re

In [12]:
# read the CSV file and then print basic info
CSV_PATH = "./data/ID_IL_movement.csv"
df = pd.read_csv(CSV_PATH)

print(f"Loaded {len(df)} rows")
print(f"Columns of the ID_IL_movement.csv: {df.columns.tolist()}")


Loaded 37667 rows
Columns of the ID_IL_movement.csv: ['injury_id', 'Date', 'Team', 'Acquired', 'Relinquished', 'Notes', 'player_name', 'player_id']


In [13]:
def extract_status(notes):
    """
    This function holds the regex that is meant to extract injury status from Notes column in injury_list.
    Returns the status or None if not found.
    
    If changes need to be made, then regex can easily be modified here. The current regex is kind of dirty but works for now.
    """
    
    notes_str = str(notes).lower()
    
    # check for when a player is out for remainder/rest of season (treat as out-for-season)
    # Check for out "for season" / "out-for-season" / "entire season" (map it to out-for-season)
    if re.search(r'remainder\s+of\s+(the\s+)?season|rest\s+of\s+(the\s+)?season', notes_str):
        return 'out-for-season'
    if re.search(r'\(?\s*out\s*[- ]for\s*season\s*\)?|entire\s+season', notes_str):
        return 'out-for-season'
    
    # Check for out indefinitely (treat as out-indefinitely)
    # Check for indefinite alone (treat as out-indefinitely) (note: this could probably be combined with above regex for 1 if statement)
    if re.search(r'\(?\s*out\s+indefinitely\s*\)?|out\s+indefinite\b(?!ly)', notes_str):
        return 'out-indefinitely'
    if re.search(r'\bindefinite\b', notes_str):
        return 'out-indefinitely'
    
    # Check for day-to-day (written out) or DTD (abbreviation) (both map to DTD)
    if re.search(r'\bday\s*[- ]to\s*[- ]day\b', notes_str, re.IGNORECASE):
        return 'DTD'
    if re.search(r'\bDTD\b', notes_str, re.IGNORECASE):
        return 'DTD'
    
    # If "months" or "month" appears in notes (map to out-for-months)
    if re.search(r'\bmonths?\b', notes_str):
        return 'out-for-months'
    
    # If "weeks" appears in notes (and not already caught above) (map to out-for-weeks)
    if re.search(r'\bweeks?\b', notes_str):
        return 'out-for-weeks'

    return None

In [14]:
# apply the extraction function
df['status'] = df['Notes'].apply(extract_status)
# save to new CSV file (THE original ID_IL_movement.csv is NOT modified!!!!)
OUTPUT_CSV_PATH = "./data/Status_IL_movement.csv"
df.to_csv(OUTPUT_CSV_PATH, index=False)
print(f"Saved new CSV to {OUTPUT_CSV_PATH}")
print(f"Columns inside Status_IL_movement.csv: {df.columns.tolist()}")


Saved new CSV to ./data/Status_IL_movement.csv
Columns inside Status_IL_movement.csv: ['injury_id', 'Date', 'Team', 'Acquired', 'Relinquished', 'Notes', 'player_name', 'player_id', 'status']


In [15]:
# Update the database
DB_PATH = "../BALL.db"

with sqlite3.connect(DB_PATH) as conn:
    # read existing data from database
    existing_df = pd.read_sql("SELECT * FROM injury_list", conn)
    
    # prep status dataframe from our processed data
    status_df = df[['injury_id', 'status']].copy()
    status_df['injury_id'] = status_df['injury_id'].astype(str)
    
    # Convert injury_id to string for merging
    existing_df['injury_id'] = existing_df['injury_id'].astype(str)
    
    # drop the old status column if it exists
    if 'status' in existing_df.columns:
        print("Dropping old 'status' column and getting the new one ready")
        existing_df = existing_df.drop(columns=['status'])
    
    # merge in the new status values
    existing_df = existing_df.merge(status_df, on='injury_id', how='left')
    
    # only clear if BOTH are missing (apply status if at least one exists)
    # THIS MEANS WE ONLY RECORD AN INJURY STATUS IF THERE'S ACTUAL INJURY DATA ALONG WITH IT
    no_injury_data = (existing_df['body_region'].isna()) & (existing_df['diagnosis'].isna())
    existing_df.loc[no_injury_data, 'status'] = None
    
    # Write back to database
    existing_df.to_sql('injury_list', conn, if_exists='replace', index=False)

# print some basic stats
print(f"\nUpdated database with status values")
print(f"Rows with status: {existing_df['status'].notna().sum()}")
print(f"Rows without status: {existing_df['status'].isna().sum()}")

# show a little table
print(f"\nStatus distribution:")
print(existing_df['status'].value_counts(dropna=False))


print("\nDatabase update complete!")

Dropping old 'status' column and getting the new one ready

Updated database with status values
Rows with status: 917
Rows without status: 36750

Status distribution:
status
None                36750
out-for-season        796
out-for-weeks          58
DTD                    43
out-indefinitely       10
out-for-months         10
Name: count, dtype: int64

Database update complete!
